In [7]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [8]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [9]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [10]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データの傾向を分析するために、以下の観点から考察します。

### 1. 商品カテゴリー別の売上分析
- **食品**、**衣服**、**日用品**の3つのカテゴリーが含まれています。それぞれのカテゴリーの売上全体を把握することが重要です。
- 各カテゴリーの売上（単価 × 数量）を集計し、どのカテゴリーが最も売上を上げているかを分析します。

### 2. 売上の時系列分析
- 売上日を基に、月ごとの売上の推移を確認します。
- 特定の時期に売上が急増または減少しているかを把握することが重要です。これは季節性やプロモーションの影響を評価するために役立ちます。

### 3. 商品パフォーマンス分析
- 各商品の売上、数量、利益（売上 - 原価）を計算します。
- 高利益商品やロングテール商品（売上が少ないが利益が高い商品）を特定します。

### 4. 客単価と在庫の効率
- 客単価（売上/購入数）を計算し、どの商品が高客単価を実現しているかを示します。
- 原価や数量を考慮し、在庫の効率や回転率を分析します。

### 5. 利益率の評価
- 各商品の利益率を計算し、高利益率の商品を特定し、パフォーマンス改善の余地を検討します。

### 6. データの可視化
- 売上の推移を折れ線グラフで視覚化し、カテゴリーごとの売上分布を円グラフや棒グラフで表示することで、より分かりやすくすることができます。

### 例: 基本的な分析
以下は各項目に対する簡単な計算例です。

- **食品**の売上合計を計算:
  - 売上合計 = Σ(単価 × 数量)

- **衣服**の売上合計を計算:
  - 売上合計 = Σ(単価 × 数量)

このような分析を行うことで、効果的なマーケティング戦略を立案し、売上の向上を図ることができます。具体的な分析結果が必要であれば、実際のデータを使用した数値計算や可視化が求められます。これらの洞察をもとに、次の施策を検討していくと良いでしょう。


In [11]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                    この売上データの傾向を分析するために、以下の観点から考察します。
1                                                    
2                                ### 1. 商品カテゴリー別の売上分析
3   - **食品**、**衣服**、**日用品**の3つのカテゴリーが含まれています。それぞれの...
4   - 各カテゴリーの売上（単価 × 数量）を集計し、どのカテゴリーが最も売上を上げているかを分...
5                                                    
6                                     ### 2. 売上の時系列分析
7                           - 売上日を基に、月ごとの売上の推移を確認します。
8   - 特定の時期に売上が急増または減少しているかを把握することが重要です。これは季節性やプロモ...
9                                                    
10                                 ### 3. 商品パフォーマンス分析
11                     - 各商品の売上、数量、利益（売上 - 原価）を計算します。
12            - 高利益商品やロングテール商品（売上が少ないが利益が高い商品）を特定します。
13                                                   
14                                   ### 4. 客単価と在庫の効率
15          - 客単価（売上/購入数）を計算し、どの商品が高客単価を実現しているかを示します。
16                       - 原価や数量を考慮し、在庫の効率や回転率を分析します。
17                          

In [12]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [13]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
